### Check data formatting
Once you have compiled a dataset and before you create a fine-tuning job, it is important to check the data formatting. To do this, we created a simple Python script which you can use to find potential errors, review token counts, and estimate the cost of a fine-tuning job.

In [3]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [4]:
data_path = "./assets/fine-tuning/annotations.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 37
First example:
{'role': 'system', 'content': 'Given an ERC token solidity function as input, return the function annotations in the format accepted by the solc-verify smart contract verifier, just this, nothing more.'}
{'role': 'user', 'content': 'function add_to_x(int n) internal;'}
{'role': 'assistant', 'content': '/// @notice precondition x == y\n/// @notice postcondition x == (y + n)\n/// @notice modifies x'}


#### Format validation
We can perform a variety of error checks to validate that each conversation in the dataset adheres to the format expected by the fine-tuning API. Errors are categorized based on their nature for easier debugging.

1. Data Type Check: Checks whether each entry in the dataset is a dictionary (dict). Error type: data_type.
2. Presence of Message List: Checks if a messages list is present in each entry. Error type: missing_messages_list.
3. Message Keys Check: Validates that each message in the messages list contains the keys role and content. Error type: message_missing_key.
4. Unrecognized Keys in Messages: Logs if a message has keys other than role, content, weight, function_call, and name. Error type: message_unrecognized_key.
5. Role Validation: Ensures the role is one of "system", "user", or "assistant". Error type: unrecognized_role.
6. Content Validation: Verifies that content has textual data and is a string. Error type: missing_content.
7. Assistant Message Presence: Checks that each conversation has at least one message from the assistant. Error type: example_missing_assistant_message.
8. The code below performs these checks, and outputs counts for each type of error found are printed. This is useful for debugging and ensuring the dataset is ready for the next steps.

In [5]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Found errors:
missing_content: 1


### Upload a training file
Once you have the data validated, the file needs to be uploaded using the Files API in order to be used with a fine-tuning jobs:

In [5]:
api_key = open("api_key.txt").read().strip()

from openai import OpenAI

client = OpenAI(api_key=api_key)

client.files.create(
  file=open("erc20_examples.jsonl", "rb"),
  purpose="fine-tune"
)


FileObject(id='file-jHmS3gdfrx8QiOHCRhTTGrZt', bytes=12889, created_at=1712811771, filename='erc20_examples.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

### Create a fine-tuned model

In [6]:
client.fine_tuning.jobs.create(
  training_file="file-jHmS3gdfrx8QiOHCRhTTGrZt", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-Q5UzrRjN624gOCEdQAE2nKBa', created_at=1712811833, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-50GaNNrNWMtO9Tsg49g78u9F', result_files=[], status='validating_files', trained_tokens=None, training_file='file-jHmS3gdfrx8QiOHCRhTTGrZt', validation_file=None, user_provided_suffix=None, seed=1167319928, integrations=[])